In [5]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [7]:
temelModel = VGG16(weights='imagenet',include_top = False,  input_shape=(224,224,3))

temelModel.summary()
for layer in temelModel.layers:
    layer.trainable = False

x = Flatten()(temelModel.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dense(2,activation='softmax')(x)

model = Model(inputs = temelModel.input, outputs = x)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [11]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss=keras.losses.binary_crossentropy, optimizer=opt,metrics=['accuracy'])

In [13]:
#Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_dataset = image.ImageDataGenerator(rescale = 1./255)

In [15]:
train_generator = train_datagen.flow_from_directory(
     "../DataSet/covid_and_normal_xray_v1/train_data",
     target_size = (224,224),
     batch_size =32,
     class_mode ='categorical')

Found 2033 images belonging to 2 classes.


In [17]:
validation_generator = test_dataset.flow_from_directory(
    '../DataSet/covid_and_normal_xray_v1/test_data',
    target_size = (224,224),
    batch_size = 32,
    class_mode ='categorical')


Found 508 images belonging to 2 classes.


In [19]:
hist = model.fit(
    train_generator,
    epochs = 10,
    validation_data = validation_generator,
)

Epoch 1/10
64/64 [==============================] - 184s 3s/step - loss: 0.0073 - accuracy: 0.9985 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 2/10
64/64 [==============================] - 184s 3s/step - loss: 0.0188 - accuracy: 0.9921 - val_loss: 0.0168 - val_accuracy: 0.9921
Epoch 3/10
64/64 [==============================] - 184s 3s/step - loss: 0.0125 - accuracy: 0.9936 - val_loss: 0.0126 - val_accuracy: 0.9921
Epoch 4/10
64/64 [==============================] - 184s 3s/step - loss: 0.0119 - accuracy: 0.9970 - val_loss: 0.0102 - val_accuracy: 0.9941
Epoch 5/10
64/64 [==============================] - 183s 3s/step - loss: 0.0052 - accuracy: 0.9980 - val_loss: 0.0115 - val_accuracy: 0.9941
Epoch 6/10
64/64 [==============================] - 183s 3s/step - loss: 0.0085 - accuracy: 0.9966 - val_loss: 0.0452 - val_accuracy: 0.9823
Epoch 7/10
64/64 [==============================] - 183s 3s/step - loss: 0.0192 - accuracy: 0.9907 - val_loss: 0.0059 - val_accuracy: 0.9980
Epoch 8/10
64

In [21]:
import matplotlib.pyplot as plt
def plot_acc_loss(hist, epochs):
    acc = hist.history['accuracy']
    loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    plt.figure(figsize=(15, 5))
    plt.subplot(121)
    plt.plot(range(1,epochs), acc[1:], label='Train_acc')
    plt.plot(range(1,epochs), val_acc[1:], label='Test_acc')
    plt.title('Accuracy over ' + str(epochs) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.subplot(122)
    plt.plot(range(1,epochs), loss[1:], label='Train_loss')
    plt.plot(range(1,epochs), val_loss[1:], label='Test_loss')
    plt.title('Loss over ' + str(epochs) +  ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
print(hist)

In [25]:
plot_acc_loss(hist,10)

<function hist at 0x7f2824a02598>


In [24]:
plot_acc_loss(hist,10)

AttributeError: 'function' object has no attribute 'history'

In [ ]:
model.save("VGG16.h5")

In [ ]:
#Load model
from tensorflow import keras
model = keras.models.load_model('../DataSet/covid_and_normal_xray_v1/model_13_03.h5')

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img=mpimg.imread('../DataSet/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/train/1-s2.0-S1684118220300682-main.pdf-003-b2.png')
imgplot = plt.imshow(img)

test_image = image.load_img('../DataSet/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/train/1-s2.0-S1684118220300682-main.pdf-003-b2.png', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

if result[0][0] == 1:
    prediction = 'normal'
else:
    prediction = 'covid'
    
#print("AI's prediction is: "+ prediction)

plt=plt.title('Prediction is  '+ prediction )
#There we will test this following image, COVID-19 positive

In [ ]:
#Prepare validation dataset
test_dataset = image.ImageDataGenerator(rescale = 1./255)

valid_generator = test_dataset.flow_from_directory(
    '../DataSet/chest-xray-for-covid19-detection/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode ='categorical',
    shuffle = False
)

In [ ]:
#See the score
score = model.evaluate(valid_generator)
print("Loss is:", score[0], "accuracy is:", score[1])

In [ ]:
import itertools 
from sklearn.metrics import classification_report, confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import seaborn as sns
%matplotlib inline

#Let's build confusion matrix
Y_pred = model.predict(valid_generator)
y_pred = np.argmax(Y_pred, axis=1)

cm = confusion_matrix(valid_generator.classes, y_pred)

class_names = ['Covid','Normal']
fig,ax = plot_confusion_matrix(conf_mat= cm, figsize=(2,2),
                              class_names = class_names,
                              show_normed = False)
plt.tight_layout()

In [ ]:
print(Y_pred)